In [1]:
import time
import os
import shutil

start = time.time()
cantidad_especies = 30
cantidad_transcritos = 4208
n_jobs = 4
verbose = 0
carpeta_base = "./real"
carpeta_data_base = carpeta_base + "/data"
carpeta_fold_base = carpeta_base + "/folds"
carpeta_modelo_base = carpeta_base + "/modelos_referenciales"
tuned_parameters = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3], 'svc__C': [0.1,0.5,0.9,2]}]
score = ['accuracy','precision','recall']

#cantidad_especies = 3
#cantidad_transcritos = 400
#carpeta_base = "./prueba"
verbose = 100

In [2]:
#if os.path.isdir(carpeta_base):
#    shutil.rmtree(carpeta_base)
#os.mkdir(carpeta_base)

In [3]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, dump

def carpeta_data():
    return carpeta_data_base

def carpeta_fold():
    return carpeta_fold_base

def carpeta_modelo():
    return carpeta_modelo_base

def obtener_especies():
    query = "SELECT * FROM (SELECT CONVERT(@row_number:=@row_number + 1, UNSIGNED) AS orden, m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie, (SELECT @row_number:=0) AS rn ORDER BY m.id_especie) a ORDER BY 1 LIMIT " + str(cantidad_especies)
    return util_bd.resultados_query(query)
        
def generar_fasta_especie(row_especie):
    os.mkdir(carpeta_data() + "/clase_" + str(row_especie[0]))
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data() + "/clase_" + str(row_especie[0]) + "/lncRNA.fa")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data() + "/clase_" + str(row_especie[0]) + "/PCT.fa")
    # CDS
    query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + str(row_especie[1]) + " ORDER BY f.cod_secuencia LIMIT " + str(cantidad_transcritos)
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, carpeta_data() + "/clase_" + str(row_especie[0]) + "/CDS.fa")

#if os.path.isdir(carpeta_data()):
#    shutil.rmtree(carpeta_data())
#os.mkdir(carpeta_data())

#%time dump(obtener_especies(), carpeta_data() + "/info_clases.bin")
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_fasta_especie)(row_especie) for row_especie in #obtener_especies())

print("Proceso finalizado")

Proceso finalizado


In [4]:
#Construir llave de folds
import os
import sys
sys.path.append('./libs')
import util_fasta
import hashlib
from sklearn.externals.joblib import dump, load
import importlib

importlib.reload(util_fasta) 

def folder_clase(num_clase):
    return carpeta_data() + "/clase_" + str(num_clase)

def archivo_clase(num_clase, tipo):
    return folder_clase(num_clase) + "/" + tipo + ".fa"

def obtener_num_clases():
    num_clases = 0
    if not os.path.isfile(carpeta_data() + "/num_clases.bin"):
        while os.path.isdir(folder_clase(num_clases + 1)):
            num_clases = num_clases + 1
        dump(num_clases, carpeta_data() + "/num_clases.bin")
    num_clases = load(carpeta_data() + "/num_clases.bin")
    return num_clases

def iterador_clases():
    return range(1, obtener_num_clases() + 1)

def obtener_primera_secuencia(num_clase):
    secuencias = util_fasta.leer_fasta(archivo_clase(num_clase, "lncRNA"), 1)
    return list(secuencias.keys())[0]

def obtener_todas_las_secuencias():
    return {num_clase : obtener_primera_secuencia(num_clase) for num_clase in iterador_clases()}

def generar_llaves_clases():
    secuencias = obtener_todas_las_secuencias()
    llaves = {num_clase : "" for num_clase in secuencias.keys()}
    llaves[0] = "" #llave cero corresponde a todo el universo
    for i_clase in iterador_clases():
        llaves[0] += secuencias[i_clase]
        for j_clase in iterador_clases():
            if (i_clase != j_clase):
                llaves[j_clase] += secuencias[i_clase]
    llaves[0] = hashlib.sha224(llaves[0].encode()).hexdigest()
    for i_clase in iterador_clases():
        llaves[i_clase] = hashlib.sha224(llaves[i_clase].encode()).hexdigest()
    dump(llaves, carpeta_data() + "/llaves_clases.bin")

def obtener_llaves_clases():
    return load(carpeta_data() + "/llaves_clases.bin")

#%time generar_llaves_clases()

print("Proceso finalizado")

Proceso finalizado


In [5]:
#Armando folds (archivos fasta)
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, load

def cargar_llaves_clases():
    return load(carpeta_data() + "/llaves_clases.bin")

def carpeta_fold_clase(llave):
    return carpeta_fold() + "/fold_clase_" + str(llave)

def archivo_fold_clase(llave, tipoTrainTest, tipoRNA):
    return carpeta_fold_clase(llave) + "/" + tipoTrainTest + "/" + tipoRNA + ".fa"

def armar_fold_final(tipo):
    llave = obtener_llaves_clases()[0]
    if not os.path.isdir(carpeta_fold_clase(llave)):
        os.mkdir(carpeta_fold_clase(llave))
    if not os.path.isdir(carpeta_fold_clase(llave) + "/train"):
        os.mkdir(carpeta_fold_clase(llave) + "/train")
    with open(archivo_fold_clase(llave, "train", tipo), "w+") as outfile:
        for num_clase in iterador_clases():
            with open(archivo_clase(num_clase, tipo), "r") as infile:
                for inline in infile:
                    outfile.write(inline)
    
def armar_fold_clase(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    os.mkdir(carpeta_fold_clase(llave))
    os.mkdir(carpeta_fold_clase(llave) + "/train")
    for tipo in ["lncRNA", "PCT", "CDS"]:
        with open(archivo_fold_clase(llave, "train", tipo), "w+") as outfile:
            for j_num_clase in iterador_clases():
                if num_clase != j_num_clase:
                    with open(archivo_clase(j_num_clase, tipo)) as infile:
                        for inline in infile:
                            outfile.write(inline)
    os.mkdir(carpeta_fold_clase(llave) + "/test")
    for tipo in ["lncRNA", "PCT"]:
        with open(archivo_fold_clase(llave, "test", tipo), "w+") as outfile:
            with open(archivo_clase(num_clase, tipo)) as infile:
                for inline in infile:
                    outfile.write(inline)
                    
#if os.path.isdir(carpeta_fold()):
#    shutil.rmtree(carpeta_fold())
#os.mkdir(carpeta_fold())

#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(armar_fold_final)(tipo) for tipo in ["lncRNA", "PCT", "CDS"])
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(armar_fold_clase)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

Proceso finalizado


In [6]:
#Generando modelos CPAT de los folds
import sys
sys.path.append('./libs')
import util_caracteristicas
import os
import importlib

importlib.reload(util_caracteristicas)

def carpeta_fold_cpat(llave):
    return carpeta_fold_clase(llave) + "/cpat"

def generar_cpat_fold(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    archivo_CDS = archivo_fold_clase(llave, "train", "CDS")
    carpeta_cpat = carpeta_fold_cpat(llave)
    if not os.path.isdir(carpeta_cpat):
        os.mkdir(carpeta_cpat)
    util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)
    
def generar_cpat_final():
    llave = obtener_llaves_clases()[0]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    archivo_CDS = archivo_fold_clase(llave, "train", "CDS")
    carpeta_cpat = carpeta_fold_cpat(llave)
    if not os.path.isdir(carpeta_cpat):
        os.mkdir(carpeta_cpat)
    util_caracteristicas.generar_modelo_CPAT(archivo_lncRNA, archivo_PCT, archivo_CDS, carpeta_cpat)

def limpieza_archivos_CDS(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    os.remove(archivo_fold_clase(llave, "train", "CDS"))

#%time generar_cpat_final()
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_cpat_fold)(num_clase) for num_clase in iterador_clases())
#%time limpieza_archivos_CDS(0)
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(limpieza_archivos_CDS)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

Proceso finalizado


In [7]:
#Ejecutar cpat y diamond sobre los folds
import os
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def ejecutar_cpat_fold(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    carpeta_cpat = carpeta_fold_cpat(llave)
    util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace(".fa", ".cpat"))
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".r")
    util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace(".fa", ".cpat"))
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".r")

def ejecutar_cpat_diamond_final():
    llave = obtener_llaves_clases()[0]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    diamond_db = "./feature_engine/Diamond_BD/uniprot-viridiplantae-reviewed.dmnd"
    carpeta_cpat = carpeta_fold_cpat(llave)
    util_caracteristicas.ejecutar_diamond(archivo_lncRNA, diamond_db, archivo_lncRNA.replace(".fa", ".dmnd"))
    util_caracteristicas.ejecutar_diamond(archivo_PCT, diamond_db, archivo_PCT.replace(".fa", ".dmnd"))
    util_caracteristicas.ejecutar_cpat(archivo_lncRNA, carpeta_cpat, archivo_lncRNA.replace(".fa", ".cpat"))
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_lncRNA.replace(".fa", ".cpat") + ".r")
    util_caracteristicas.ejecutar_cpat(archivo_PCT, carpeta_cpat, archivo_PCT.replace(".fa", ".cpat"))
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".dat")
    os.remove(archivo_PCT.replace(".fa", ".cpat") + ".r")

#%time ejecutar_cpat_diamond_final()
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(ejecutar_cpat_fold)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

Proceso finalizado


In [8]:
#Generar archivo de features por cada fold
import os
import sys
sys.path.append('./libs')
import util_caracteristicas
import importlib

importlib.reload(util_caracteristicas)

def archivo_features_clase(llave, tipoTrainTest, tipoRNA):
    return archivo_fold_clase(llave, tipoTrainTest, tipoRNA).replace(".fa", ".ft")

def generar_features_fold(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    features_base_lncRNA = archivo_features_clase(obtener_llaves_clases()[0], "train", "lncRNA")
    features_base_PCT = archivo_features_clase(obtener_llaves_clases()[0], "train", "PCT")
    for tipo in ["train", "test"]:
        archivo_lncRNA = archivo_fold_clase(llave, tipo, "lncRNA")
        archivo_PCT = archivo_fold_clase(llave, tipo, "PCT")
        archivo_cpat_lncRNA = archivo_fold_clase(llave, "train", "lncRNA").replace(".fa", ".cpat")
        archivo_cpat_PCT = archivo_fold_clase(llave, "train", "PCT").replace(".fa", ".cpat")
        salida_lncRNA = archivo_features_clase(llave, tipo, "lncRNA")
        salida_PCT = archivo_features_clase(llave, tipo, "PCT")
        util_caracteristicas.generar_features(archivo_lncRNA, features_base_lncRNA, archivo_cpat_lncRNA, salida_lncRNA)
        util_caracteristicas.generar_features(archivo_PCT, features_base_PCT, archivo_cpat_PCT, salida_PCT)
    
def generar_features_final():
    llave = obtener_llaves_clases()[0]
    archivo_lncRNA = archivo_fold_clase(llave, "train", "lncRNA")
    archivo_PCT = archivo_fold_clase(llave, "train", "PCT")
    salida_lncRNA = archivo_features_clase(llave, "train", "lncRNA")
    salida_PCT = archivo_features_clase(llave, "train", "PCT")
    util_caracteristicas.generar_features_base(archivo_lncRNA, archivo_lncRNA.replace(".fa", ".cpat"), archivo_lncRNA.replace(".fa", ".dmnd"), salida_lncRNA)
    util_caracteristicas.generar_features_base(archivo_PCT, archivo_PCT.replace(".fa", ".cpat"), archivo_PCT.replace(".fa", ".dmnd"), salida_PCT)

#%time generar_features_final()
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(generar_features_fold)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")

Proceso finalizado


In [ ]:
#Generar modelo final
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV
from sklearn.externals.joblib import dump, load
import hashlib
import util_fasta
import os
from sklearn.preprocessing import RobustScaler
import importlib

importlib.reload(util_fasta)

def obtener_data_entrenamiento():
    llave = obtener_llaves_clases()[0]
    codigos_lncRNA = util_fasta.leer_fasta_list(archivo_fold_clase(llave, "train", "lncRNA"))
    codigos_PCT = util_fasta.leer_fasta_list(archivo_fold_clase(llave, "train", "PCT"))
    
    codigos_lncRNA = [(x[0],"") for x in codigos_lncRNA]
    codigos_PCT = [(x[0],"") for x in codigos_PCT]
    
    y = ([1] * len(codigos_lncRNA)) + ([0] * len(codigos_PCT))
    groups = list()
    for _ in range(2):
        for num_clase in iterador_clases():
            groups += ([num_clase] * (cantidad_transcritos))
    return codigos_lncRNA + codigos_PCT, y, groups

class GeneradorFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, cantidad_transcritos=None, num_clases=None):
        if cantidad_transcritos is None:
            return
        self.cantidad_transcritos = cantidad_transcritos
        self.num_clases = num_clases
        
    def fit(self, X, y=None):
        self._llave_fold = self.obtener_llave_fold(X)
        return self
        
    def transform(self, X):
        return self.obtener_features_pre_calculados(X)
    
    def obtener_llave_fold(self, X):
        cod_secuencias = ""
        cod_secuencias2 = ""
        num_transcritos = len(X)
        num_transcritos_por_grupo = self.cantidad_transcritos
        for i in range(num_transcritos//(num_transcritos_por_grupo*2)):
            cod_secuencias += X[i * num_transcritos_por_grupo][0]
            cod_secuencias2 += X[i * num_transcritos_por_grupo][0] + "*****"
        llave = hashlib.sha224(cod_secuencias.encode()).hexdigest()
        return llave
    
    def obtener_features_pre_calculados(self, X):
        llave = self._llave_fold
        tipo = "train"
        if os.path.isfile(archivo_fold_clase(llave, "test", "lncRNA")):
            secuencias = util_fasta.leer_fasta(archivo_fold_clase(llave, "test", "lncRNA"), 1)
            if list(secuencias.keys())[0] == X[0][0]:
                tipo = "test"
        features = list(load(archivo_features_clase(llave, tipo, "lncRNA")).values())
        features += list(load(archivo_features_clase(llave, tipo, "PCT")).values())
        return [list(x.values()) for x in features]

def generar_modelo_final(tuned_parameters):
    X_train, y_train, groups = obtener_data_entrenamiento()
    svm_pipeline = Pipeline(steps=[('features', GeneradorFeatures(cantidad_transcritos, obtener_num_clases())), ('svc', SVC())])
    logo = LeaveOneGroupOut()
    clf = GridSearchCV(svm_pipeline, tuned_parameters, cv=logo, scoring=score, n_jobs=n_jobs, refit="accuracy", return_train_score = True, verbose=verbose)
    clf.fit(X_train, y_train, groups) #requerido por LeaveOneGroupOut
    resultado = {
        "accuracy" : clf.cv_results_['mean_test_accuracy'][clf.best_index_],
        "precision" : clf.cv_results_['mean_test_precision'][clf.best_index_],
        "recall" : clf.cv_results_['mean_test_recall'][clf.best_index_]
    }
    dump(resultado, carpeta_modelo() + "/resultado.bin")
    dump(clf.best_params_, carpeta_modelo() + "/params.bin")
    dump(clf.cv_results_, carpeta_modelo() + "/cv_results.bin")
    dump(clf.best_estimator_, carpeta_modelo() + "/modelo.plk")

def prueba_de_modelo_final():
    X_train, y_train, groups = obtener_data_entrenamiento()
    svm_pipeline = Pipeline(steps=[('features', GeneradorFeatures(cantidad_transcritos, obtener_num_clases())), ('svc', SVC(kernel="rbf",gamma=1e-3,C=2,verbose=verbose))])
    svm_pipeline.fit(X_train, y_train)
    dump(svm_pipeline, carpeta_modelo() + "/modelo2.plk")
    
if not os.path.isdir(carpeta_modelo()):
    #shutil.rmtree(carpeta_modelo())
    os.mkdir(carpeta_modelo())

%time prueba_de_modelo_final()
#%time generar_modelo_final(tuned_parameters)
    
print("Proceso finalizado")

[LibSVM]

In [ ]:
import shutil
from sklearn.externals.joblib import Parallel, delayed

def limpieza_archivos_finales_fasta_ruta(llave):
    shutil.rmtree(carpeta_fold_clase(llave))
    
def limpieza_archivos_finales_fasta(num_clase):
    llave = obtener_llaves_clases()[num_clase]
    if num_clase == 0:
        shutil.rmtree(carpeta_fold_clase(llave) + "/train")
    else:
        limpieza_archivos_finales_fasta_ruta(llave)
        
#%time limpieza_archivos_finales_fasta(0)
#%time Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(limpieza_archivos_finales_fasta)(num_clase) for num_clase in iterador_clases())
    
print("Proceso finalizado")